In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

In [2]:
df_cases = pd.read_csv('../datasets/data_list.txt', sep='\t')
cases = df_cases.data_id.to_list()
df_cases.index = df_cases.data_id.to_list()

In [3]:
df_compile = pd.DataFrame(columns=cases)

for case in cases:
    path_data = '../datasets/' + case + '/'
    biom_id = df_cases.biomass_dilution_id[case]
    mu = df_cases.growth_rate[case]
    
    # Get kapp (1/s)
    path = path_data + 'kapps_in_vivo.txt'
    df_kapp = pd.read_csv(path, sep='\t')
    
    for i in df_kapp.index:
        _,rxn,rdir,enz = extract_details_from_rxnid(df_kapp.rxnid[i])
        regid = rxn + '_' + rdir
        df_compile.loc[regid, case] = df_kapp.loc[i,'kapp (1/s)']

In [10]:
df_compile['id'] = df_compile.index.to_list()

In [12]:
df_rxnenz = pd.read_excel('./input/rxn_enz_stage1.xlsx')
for i in df_rxnenz.index:
    rxn = df_rxnenz.rxn_id[i]
    rdir = df_rxnenz.direction[i]
    entry = rxn + '_' + rdir
    if entry in df_compile.index:
        df_compile.loc[entry, 'subsystem_group'] = df_rxnenz.loc[i, 'subsystem_group']
        df_compile.loc[entry, 'subsystem'] = df_rxnenz.loc[i, 'subsystem']

In [14]:
idx_excl = ['ACALDCD_c_FWD']
idx = [i for i in df_compile.index if i not in idx_excl]
df_compile = df_compile.loc[idx,:]

In [15]:
cols = df_compile.columns.to_list()
df_compile = df_compile.loc[:, cols[-3:] + cols[:-3]]

In [16]:
df_compile.to_excel('../results/kapp_inPerSec_frompy.xlsx', index=None)

#### Compile pathway to rxn-enz entries

In [39]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

In [44]:
df_pw = pd.read_excel('./input/GSM_pathway_assignment.xlsx')
df_pw.index = df_pw.rxnid.to_list()

In [45]:
df_rxnenz = pd.read_excel('./input/rxn_enz_stage1.xlsx')

In [46]:
for i in df_rxnenz.index:
    rxn = df_rxnenz.rxn_id[i]
    df_rxnenz.loc[i,'subsystem_group'] = df_pw.loc[rxn,'subsystem_group']
    df_rxnenz.loc[i,'subsystem'] = df_pw.loc[rxn,'subsystem']

In [49]:
df_rxnenz.to_excel('./input/rxn_enz_stage1_frompy.xlsx', index=None)

In [12]:
import cobra

In [13]:
model = cobra.io.load_json_model('../../scRBA_build_GAMS_model/input/GSM_y834_hvd_v6_rba.json')

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2022-08-08
